# For a single product, Hexowatch, in Data Analysis Category 

In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup
import re 
import pandas as pd
import json 
import csv 
from urllib.parse import urljoin

In [2]:
url = 'https://www.capterra.com/p/206900/Hexowatch/'

In [3]:
driver = webdriver.Chrome(r'C:\Users\HP\Downloads\chromedriver.exe')  
driver.get(url)

In [4]:
soup = BeautifulSoup(driver.page_source,"html.parser" )

In [5]:
soup_obj = soup.find('div',{'class':'ProductSummary__SummaryPanel-mcm4i7-6 hCwGXf'})

In [6]:
product = soup_obj.contents[0]
product_info = product.select('div > div')[0].get_text(strip=True) #this is info about the product 
print(product_info)
#for featured in 
x = soup_obj.contents[1].h2.get_text()

if re.match(r"Featured In", x):
    c = soup_obj.contents[2]
    d = c.select('div > em')[0].get_text(strip=True)
    print(d) #best for
    
    e = soup_obj.contents[3]
    f = e.select('div > p')[0].get_text(strip=True)
    print(f) # company name
    
    g = soup_obj.contents[3]
    h = g.select('div > p')[1].get_text(strip=True)
    print(h) # product website 
    
    i = soup_obj.contents[3]
    j = i.select('div > p')[2].get_text(strip=True)
    if not j:
        print('NaN')
    else:
        print(j) # founding year 
        
    k = soup_obj.contents[3]
    l = k.select('div > p')[3].get_text(strip=True)
    if not j:
        print('NaN') # founding place
    else:
        print(l)
        
else:
    c = soup_obj.contents[1]
    d = c.select('div > em')[0].get_text(strip=True)
    print(d)
    
    e = soup_obj.contents[2]
    f = e.select('div > p')[0].get_text(strip=True)
    print(f)
    
    g = soup_obj.contents[2]
    h = g.select('div > p')[1].get_text(strip=True)
    print(h)
    
    i = soup_obj.contents[2]
    j = i.select('div > p')[2].get_text(strip=True)
    if not j:
        print('NaN')
    else:
        print(j)
    
    k = soup_obj.contents[2]
    l = k.select('div > p')[3].get_text(strip=True)
    if not l:
        print('NaN')
    else:
        print(l)
info = {'About Product': product_info,'Best For':d,'Company Name': f,'Product Website': h, 'Founding Year': j,'Founding Place': l }

Hexowatch is your AI sidekick to monitor any website for visual, content, source code, technology, availability or price changes.
Hexowatch works 24/7 to help spot trends, spy on your competitors, visually check your website, keep an archive of every change and turn any web page into your own private data source - Get started in minutes - no software, proxies or programming required
For Business and Personal use
Hexact
http://hexometer.com
Founded in 2019
Located in United States


# Another Part of the page - The features & their buttons

In [7]:
new_soup = soup.find('div',{'class':'SpecTable__Root-sc-1jq5uy4-0 jZHgUG'})

In [8]:
# the listed price of the product, if given.
price_heading = new_soup.contents[0].h3.get_text()
price = new_soup.contents[0].a.get_text()
listed_price = {price_heading:price}
listed_price

{'Starting Price': '$14.99/month'}

In [9]:
# for available features in Deployment, Training,& Support
ndic ={}
for i in range(1,4):
    s_obj = new_soup.contents[i]
    heading = s_obj.h3.get_text()
    new_obj = s_obj.contents[1]
    nl = []
    for i in range(len(new_obj)):
        an_obj = new_obj.contents[i]
        x = an_obj.select('div > div')[0]['class']
        y= x[1]
        if y == 'bqMKDC':
            q = an_obj.select('div > div')[1].get_text(strip=True)
            nl.append(q)
    
    ndic[heading] =nl
    
ndic

{'Deployment': ['Web-Based, Cloud, SaaS'],
 'Training': ['Live Online', 'Documentation'],
 'Support': ['24/7 (Live Rep)', 'Business Hours', 'Online']}

# Buttons

Clicking all the field buttons available for the product and clicking it and extracting corresponding features in each field.


In [10]:
another_soup_obj = soup.find('div',{'class':'Accordion__Root-sc-1jo79it-0 jGWhUg'})

In [11]:
# clicking all the buttons 
for i in range(1,len(another_soup_obj.contents)):
    h4 = another_soup_obj.contents[i].h4.get_text()
    print(f'"{h4}"')
    cb = "//button[normalize-space()=" + f'"{h4}"' + "]"
    element = driver.find_element_by_xpath(cb)
    driver.execute_script("arguments[0].click();", element)

In [12]:
# extracting the corresponding features
nd = {}
for i in range(len(another_soup_obj)):
    field_heading = another_soup_obj.contents[i].h4.get_text()
    n_o_j = another_soup_obj.contents[i]
    xyz = n_o_j.find_all('div',{'class':'FeatureList__FeatureCell-sc-1kxeq27-1 ifqTCQ'})
    nl = []
    for i in xyz:
        zas = i.select('div > div')[1]['class']
        sac = zas[1]
        if sac == 'bqMKDC':
             q = i.select('div > div')[2].get_text(strip=True)
             nl.append(q)
    nd[field_heading] = nl
    
nd

{'Data Analysis': ['Dashboard',
  'Data Discovery',
  'Data Visualization',
  'Reporting/Analytics']}

# Now scraping the reviews from the reviews page 

In [13]:
newurl = urljoin(url,'reviews')

In [14]:
another_driver = webdriver.Chrome(r'C:\Users\HP\Downloads\chromedriver.exe')
another_driver.get(newurl)

Now we click the show more reviews button till we get all the reviews in a single page

In [15]:
while True:
    try:
        new_element = another_driver.find_element_by_xpath('//button[normalize-space()="Show more reviews"]')
        another_driver.execute_script("arguments[0].click();", new_element)
    except:
        break

In [16]:
review_soup = BeautifulSoup(another_driver.page_source,"html.parser" )

# Overall Ratings

Here we'll get the overall ratings for the product.

In [17]:
ls = ['Overall','Ease of Use','Customer Service']
ratings = soup.find_all('div',{'class':'StarRating__Rating-sc-9jwzgg-1 cAGyvf'})
la = []
for x,y in zip(ls,ratings):
    la.append({x:y.get_text(strip=True)})
    
la

[{'Overall': '4.6/5'}, {'Ease of Use': '4.4/5'}, {'Customer Service': '4.6/5'}]

# Reviewer Info

In [18]:
start = review_soup.find('div',{'class':'gtm-review-section'})

In [24]:
empty_list = []

for i in range(len(start.contents)):

#reviewer info
    qw = start.contents[i]
    personal_info = qw.find('div',{'class':'ReviewerAvatarSection__ReviewerInfo-sc-1a80501-1 cSadMy'})
    name = personal_info.contents[0].get_text(strip=True)
    other_info = personal_info.contents[1]
    designation = other_info.select('div > div')[0].get_text(strip=True)
    working_domain = other_info.select('div > div')[1].get_text(strip=True) 
    usage = other_info.select('div > div')[2].get_text(strip=True)
    reviewer_info = {name:[designation,working_domain,usage]}
    empty_list.append(reviewer_info)

#reviewer ratings    
    li = ['Overall','Ease of Use','Customer Service', 'Features', ' Value for Money','Likelihood to Recommend']
    ra = qw.find_all('div',{'class':'StarRating__Rating-sc-9jwzgg-1 kCLiav'})
    reviewer_ratings = []
    for x,y in zip(li,ra):
        reviewer_ratings.append({x:y.get_text(strip=True)})
    empty_list.append(reviewer_ratings)

#date    
    date = qw.find('div',{'class':'ReviewSource__Root-lnjke6-0 iBAGLQ'}).contents[2].get_text(strip=True)
    empty_list.append(date)

#review

    review = qw.find('div',{'class':'ReviewCard__RightFlexItem-sc-18j15p9-3 cOmWnW ReviewCard__FlexItem-sc-18j15p9-2 bTVfYT'})
    review_heading = review.contents[0].h3.get_text(strip=True)
    overall = review.contents[0].find_all('div',{'class':'ReviewSection__Root-sc-189472c-0 icjcMH'})
    review_text ={}
    review_text['review heading'] = review_heading
    for i in overall :
        text = i.get_text(strip=True)
        manip = text.split(':')
        header = manip[0]
        review_content = manip[1]
        review_text[header] = review_content
        
    
    empty_list.append(review_text)
    
    

    
empty_list

[{'Richard v.': ['Technical Support Engineer',
   'Online Media, 1-10 employees',
   'Used the software for: 6-12 months']},
 [{'Overall': '5/5'},
  {'Ease of Use': '5/5'},
  {'Customer Service': '5/5'},
  {'Features': '5/5'},
  {' Value for Money': '5/5'}],
 'September 25, 2020',
 {'review heading': '“tip of the iceberg”',
  'Overall': 'So far a great experience. In the beginning some pages made the google chrome tab freeze, I reported this bug and it was solved really quickly!',
  'Pros': 'I bought Hexowatch purely for the visual monitoring. But it has so much more monitoring options that it has its value in almost any use-case. But the amount of checks/month for their current price point made it an absolute no brainer',
  'Cons': "It's still in a early stage so I wasn't to happy to manually add each monitor but, it's currently on the roadmap to make that easier.",
  'Alternatives Considered': 'ChangeTowerandGhost Inspector',
  'Reasons for Choosing Hexowatch': 'user to setup and use

In [25]:
with open('review_data1.csv', mode='w') as csv_file:
    fieldnames = ['Product Info', 'Price', 'Overall Features','Category Features', 'Overall Ratings', 'Review']
    writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
    writer.writeheader()
    writer.writerow({'Product Info': info , 'Price': listed_price, 'Overall Features': ndic ,'Category Features': nd ,'Overall Ratings': la, 'Review': empty_list})

In [22]:
another_driver.close()

In [23]:
driver.close()